In [1]:
# !pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus
# !pip install plotly

# Example 1 - Atlantic county, using the previous R2D inputs

In [2]:
# Written: sy Aug 2024
# License: BSD-2

"""
 Purpose: Testing Imputer
"""

import os
import sys
import copy
import json

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


## Loading existing inventory

In [3]:
file_path = "./nsi_altantic_minimal.csv"

In [4]:
inventory = AssetInventory()
inventory.read_from_csv(file_path,keep_existing=True,id_column='id') 

No existing inventory found. Reading in the new inventory from the file.


True

In [5]:
# Inspecting the features of asset #4
inventory.get_asset_features(4)[1]  # empty or 'NA' are missing values

{'id': 4,
 'City': 'Somers Point',
 'OccupancyClass': 'RES1',
 'BuildingType': 'Wood',
 'DesignLevel': 'NE',
 'YearBuilt': 1994,
 'NumberOfStories': 2,
 'PlanArea': 2416,
 'FoundationType': 3504,
 'SplitLevel': 0,
 'HasGarage': 0,
 'FirstFloorHt': 2,
 'FloodZone': 'X',
 'WindZone': 'I',
 'AvgJanTemp': 'Above',
 'RoofShape': 'Hip',
 'RoofSlope': 0,
 'RoofSystem': 'Truss',
 'WindowArea': 0,
 'DesignWindSpeed': 121.5296499,
 'NumberOfUnits': 1,
 'Height': 35.04684071,
 'LandCover': 'Trees',
 'OBJECTID': 847181,
 'geometry': 'POINT (-74.61723667499923 39.30829801705697)',
 'type': 'building'}

In [6]:
#inventory.remove_features(['RoofShape'])

## Importing inference engine and inferring hurricane wind features

In [7]:
importer = Importer()
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer=hurricaneInferer(input_inventory=inventory,clean_features = True)

In [8]:
new_inventory = inferer.infer()

Done inference. It took 0.38 mins


In [9]:
new_inventory.get_asset_features(156)[1]  # empty or 'NA' are missing values

{'BuildingType': 'Wood',
 'StructureType': 'Single Family Housing',
 'LandCover': 'Trees',
 'NumberOfStories': 1,
 'RoofShape': 'Gable',
 'SecondaryWaterResistance': 1,
 'Shutters': 1,
 'RoofDeckAttachment': '8d',
 'RoofToWallConnection': 'Strap',
 'Garage': 'No'}

In [10]:
_ = new_inventory.write_to_geojson(
    output_file="nsi_altantic_minimal.json")

# Example 2 - Fort Myers Beach, FL

In [11]:
LOCATION_NAME = 'Fort Myers Beach, FL' #'Fort Myers Beach, FL'
INVENTORY_OUTPUT = 'FortMyersInventory_HU.geojson' #'FortMyersInventory_HU.geojson'
NO_POSSIBLE_WORLDS = 1

In [12]:
importer = Importer()

### Merging NSI and footprint

In [13]:
region_data = {"type": "locationName", "data": LOCATION_NAME}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [14]:
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States


INFO:root:
Getting National Structure Inventory (NSI) building data for the entered location...



Found a total of 3558 building points in NSI that are within the entered region of interest


In [15]:
scraper_class = importer.get_class('USA_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)


Searching for Fort Myers Beach, FL...
Found Fort Myers Beach, Lee County, Florida, 33931, United States

Meshing the defined area...


Obtaining the number of elements in each cell: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.06s/it]



Meshing complete. Split Fort Myers Beach into 10 cells


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 11437.97it/s]



Found a total of 3102 building footprints in Fort Myers Beach


In [16]:
nsi_inventory = nsi.get_filtered_data_given_inventory(
    scraper_inventory, "ft", get_extended_features=True)


Getting National Structure Inventory (NSI) building data for the entered location...
Found a total of 2796 building points in NSI that match the footprint data.


### Running imputation

In [17]:
knn_imputer_class = importer.get_class("KnnImputer")

imputer = knn_imputer_class(
    nsi_inventory, n_possible_worlds=NO_POSSIBLE_WORLDS,
    exclude_features=['lat', 'lon', 'fd_id'])
imputed_inventory = imputer.impute()

Existing worlds: 1
New worlds per existing world: 1
world # 0
Features with no reference data cannot be imputed. Removing them from the imputation target: buildingheight
Missing percentages among 3102 assets
fparea: 9.86%
numstories: 9.86%
erabuilt: 9.86%
repaircost: 9.86%
constype: 9.86%
occupancy: 9.86%
found_ht: 9.86%
splitlevel: 9.86%
basement: 28.11%
Primitive imputation done.
Running the main imputation. This may take a while.
Done imputation. It took 0.01 mins


### Temporarily Assigning features 
in the future, these features should be provided by the brails scraper

In [18]:
for key, val in imputed_inventory.inventory.items():
    val.add_features({"DesignWindSpeed":30,
                     "RoofShape":'Hip',
                     "AvgJanTemp":'Above',
                     "LandCover":'Trees'})

In [19]:
imputed_inventory.change_feature_names({'erabuilt': 'YearBuilt',
                                        'constype': 'BuildingType',
                                        'fpAreas': 'PlanArea',
                                        'numstories': 'NumberOfStories',
                                        'occupancy': 'OccupancyClass'
                                       })
hurricaneInferer = importer.get_class("HazusInfererWind")
inferer = hurricaneInferer(input_inventory=imputed_inventory, clean_features=True,n_possible_worlds=NO_POSSIBLE_WORLDS)
hazus_inferred_inventory = inferer.infer()

Done inference. It took 0.01 mins


### Checking the Outcomes

In [20]:
hazus_inferred_inventory.inventory[27].features